In [5]:
import os
import scipy.io as sio
import numpy as np

In [7]:
# Import all .mat files in Depression_Study\export_mat\ folder

data = {}

def import_mat_files(folder_path):
    """
    Import all .mat files from the specified folder and return a dictionary
    with the file names as keys and the loaded data as values.
    
    :param folder_path: Path to the folder containing .mat files
    :return: Dictionary with file names as keys and loaded data as values
    """
    mat_files = [f for f in os.listdir(folder_path) if f.endswith('.mat')]
    data_dict = {}
    
    for file_name in mat_files:
        file_path = os.path.join(folder_path, file_name)
        data = sio.loadmat(file_path)
        data_dict[file_name] = data
    
    return data_dict

folder_path = 'Depression_Study/export_mat/'
if not os.path.exists(folder_path):
    print(f"Folder {folder_path} does not exist.")
else:
    data = import_mat_files(folder_path)
    print(f"Imported {len(data)} .mat files from {folder_path}.")

MemoryError: 

In [2]:
print(data)  # Print the imported data dictionary

{'507_Depression_REST_processed.mat': {'__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Sat May  3 17:51:22 2025', '__version__': '1.0', '__globals__': [], 'epochs_open': array([[[-4.64471483e-08,  1.44504598e-08,  3.96868063e-06, ...,
          3.83640020e-08, -6.41854461e-06, -8.47707348e-06],
        [ 2.53593615e-08,  3.82958441e-06,  7.57311914e-06, ...,
         -9.07945106e-07, -3.94442869e-06, -3.74719814e-06],
        [-2.36941029e-08,  4.94163132e-06,  6.33369870e-06, ...,
         -4.11883606e-06, -5.10155315e-06, -2.99492113e-06],
        ...,
        [-5.29694844e-10, -1.52262319e-05, -1.96785945e-05, ...,
         -5.17301861e-06, -4.50286363e-06, -1.72916215e-06],
        [-1.04664720e-03, -1.04664522e-03, -1.04664359e-03, ...,
         -8.00201611e-04, -8.28066783e-04, -8.50425917e-04],
        [-2.56829167e-04, -2.55585473e-04, -2.52317579e-04, ...,
          6.60533697e-05,  6.34731696e-05,  6.42023588e-05]],

       [[ 7.07970260e-07,  2.54693279e-06,  2

In [ ]:
# Plot power spectral density (PSD) for 